![](https://lh3.googleusercontent.com/proxy/9c4hGTphlM6oF-ZLuK0X4g6rsEH2bDMXW4fo9tWLpUAneJersmRFs14cbrM0TQbJsDVIO6EIbu9etambTn3zLvlkvfYJgU5z5uzkPIkV-teGFh4Tw680-o6PIyVP8LHwi8IjaCJzRJWTrrd88YN9NmM)

# **Bruno Dutra e Diogo Ceddia**
# **Inteligência Artificial** 
# **Trabalho 3 - Aprendizado de Máquina**


Inicialmente, esse dataset fora escolhido devido ao seu tamanho (quase um milhão de linhas e 45 colunas). Dessa forma, é possível que possamos filtrar e preprocessar à vontade, sem que haja preocupação em reduzir demasiadamente o dataset. Inicialmente, o dataset foi divulgado com o intuito de realizar a predição do diâmetro do asteróide. Entretanto, foi de interesse da dupla proceder predição classificatória, e não regressiva, visto que nossa pouca experiência somente contemplou análise regressiva. Buscamos implementar algo que nunca haviamos tentado implementar.

https://www.kaggle.com/basu369victor/prediction-of-asteroid-diameter/tasks

# 1 - Inicialização do modelo

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('/kaggle/input/asteroid-dataset/dataset.csv', low_memory=False)

pd.set_option('display.max_columns', 500)

print(f'Quantidade de linhas da matriz: {data.shape[0]} \nQuantidade de colunas na matriz: {data.shape[1]}')

Para reduzir o tamanho da matriz, assim como definir o escopo da predição:

In [ ]:
data = data[data['class'].isin(['IMB', 'MCA', 'APO', 'AMO', 'TJN', 'TNO'])].reset_index(drop=True)

print(f'Quantidade de linhas da matriz: {data.shape[0]} \nQuantidade de colunas na matriz: {data.shape[1]}')
data.head()

Deletando colunas de ID/strings de identificação que não são úteis para predição.

In [ ]:
data = data.drop(['id','spkid','full_name','name','orbit_id','equinox','pdes','prefix'],axis=1)


data.head()

Identificando colunas que tem muitos valores faltando, colunas com mais de 80% serão removidas pois não tem como tratar esses valores de forma razoável.

In [ ]:
total_rows = data.shape[0]
missing_values_columns = [];

print("Colunas a serem removidas: \n")

for column in data:
    
    not_na = (1 - (data[column].count() / total_rows)) * 100
    
    if(not_na > 80):
        missing_values_columns.append(column)
        print(column,': %.2f' % not_na)
    else:
        data = data[data[column].notna()]
        
data = data.drop(missing_values_columns, axis='columns', inplace=False) 


In [ ]:
dataInfo = data.shape

print('Quantidade de Linhas: ', dataInfo[0])
print('Quantidade de Colunas: ', dataInfo[1])

# 2 - Pré-Processamento

Identificando as colunas que não estão representadas de forma númerica e vão precisar ser categorizadas para serem entendidas pelo modelo.

In [ ]:
categorial_columns = []
numerical_columns = []

for column in data:
    if(data[column].dtypes != "float64" and data[column].dtypes != "int64"):
        categorial_columns.append(column)
    else:
        numerical_columns.append(column)

# Removendo a coluna 'class' pois o modelo irá predizer esse valor
categorial_columns.remove('class')

print("Colunas a serem categorizadas: ", categorial_columns)
print("Colunas a serem scaladas: ", numerical_columns)

Realização do data split, utilizando 10% para treinamento e 90% para validação. Como o dataset é desbalanceado, utilizamos o comando stratity para realizar uma amostragem estratificada proporcional.

In [ ]:
from sklearn.model_selection import train_test_split

#Separando dataset de teste

X=data.drop(['class'], axis=1)
y=data['class']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.1, stratify=y)

Definição das métricas para avaliação do desempenho dos modelos

In [ ]:
import warnings
from sklearn import metrics 

warnings.filterwarnings('always')

def metricCalculation(classifier, y_test, pred, best_params):
    
    precision_metric = metrics.precision_score(y_test, pred, average = "macro")
    recall_metric = metrics.recall_score(y_test, pred, average = "macro")
    accuracy_metric = metrics.balanced_accuracy_score(y_test, pred)
    f1_metric = metrics.f1_score(y_test, pred, labels=np.unique(pred), average = "macro")

    return {
        'classifier': str(classifier).split('(')[0],
        'precision': round(precision_metric, 2),
        'recall': round(recall_metric, 2),
        'accuracy': round(accuracy_metric, 4),
        'f1-score': round(f1_metric, 2)
    }

# 3 - Execução Pipeline

Nessa etapa, elencamos 4 modelos relevantes para comparação:

**DecisionTreeClassifier**: que é somenteuma árvore de decisão classificatória;

**ExtraTreeClassifier**: que é similar ao RandomForest, porém possivelmente mais rápido computacionalmente e insere possivelmente mais ruído na predição;

**RandomForestClassifier**: que consiste na utilização de várias árvores de decisão simultaneamente;

**XGBClassifier**: que são árvores de decisão com gradiente aumentado projetadas para velocidade e desempenho.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

from operator import itemgetter

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay

# Compondo os pré-processadores

preprocessor = ColumnTransformer(transformers=[
    ('scaler', MinMaxScaler(), numerical_columns),
    ('one-hot', OneHotEncoder(sparse = False), categorial_columns)    
])

#Classifier Parameters

parameters = [ 
                { 
                    'clf': [DecisionTreeClassifier()],
                    'clf__max_depth': [None, 3, 4, 5], 
                    'clf__criterion': ['gini', 'entropy'],
                    'clf__min_samples_split': [None, 100, 1000, 10000],
                    'clf__max_features': [ None , "sqrt", "log2"],
                    'clf__class_weight': [None, "balanced"]
                    
                },{ 
                    'clf': [ExtraTreeClassifier()],
                    'clf__max_depth': [None, 3, 4, 5], 
                    'clf__criterion': ['gini', 'entropy'],
                    'clf__min_samples_split': [None, 100, 1000, 10000],
                    'clf__max_features': [ None , "sqrt", "log2"],
                    'clf__class_weight': [None, "balanced"]
                },{
                    'clf': [RandomForestClassifier()],
                    'clf__random_state': [None, 100, 100, 1000],
                    'clf__criterion': ['gini', 'entropy']
                }, {
                    'clf': [XGBClassifier()]
                }

]

result=[]
metrics_result=[]

for params in parameters:

    
    #classifier
    clf = params['clf'][0]

    #getting arguments by
    #popping out classifier
    params.pop('clf')

    #pipeline
    steps = [
                ('preprocessor', preprocessor), 
                ('clf', clf)
    ]
    
    kfold = KFold(n_splits=3, shuffle=True)

    grid = GridSearchCV(Pipeline(steps), param_grid=params, cv=kfold, n_jobs=-1, refit=True)
    grid.fit(X_train, y_train)

    y_pred = grid.best_estimator_.predict(X_valid)
    
    metrics_result.append(metricCalculation(clf, y_valid, y_pred, grid.best_params_))
    
    #storing result
    result.append({
                'grid': grid,
                'classifier': grid.best_estimator_,
                'best score': grid.best_score_,
                'best params': grid.best_params_,
                'cv': grid.cv
    })

#sorting result by best score
best_result = sorted(result, key=itemgetter('best score'),reverse=True)

#saving best classifier
best_grid = best_result[0]['grid']

# 4 - Métricas

In [ ]:
m_results = pd.DataFrame(metrics_result)

m_results = m_results.sort_values(by=['precision',
                                      'accuracy',
                                      'recall',
                                      'f1-score'
                                     ], ascending=False)

m_results[0:6]

Portanto, analisamos que os modelos convergem em acurácia, e as features são capazes de explicar com muita precisão o target.

# 5 - Matrix de Confusão

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

fig, axn = plt.subplots(2, 2, figsize=(30, 8))

plt.subplots_adjust(top=12, bottom=10)

for i, ax in enumerate(axn.flat):
    k = result[i]
    
    estimator_ = k['grid'].best_estimator_
    
    plot_confusion_matrix(estimator_ , X_train, y_train).plot(ax=ax)
    ax.set_title(str(estimator_['clf']).split('(')[0] ,fontsize=12)
    
    plt.close()
    
plt.show()

# 6 - Curva ROC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.preprocessing import label_binarize

fig, axn = plt.subplots(2, 2, figsize=(30, 8))

plt.subplots_adjust(top=12, bottom=10)

for i, ax in enumerate(axn.flat):
    k = result[i]
    
    estimator_ = k['grid'].best_estimator_
    
    y_pred_ = estimator_.predict(X_valid)
    
    y_pred_ = label_binarize(y_pred_, classes = estimator_.classes_)
    y_valid_ = label_binarize(y_valid, classes = estimator_.classes_)
    
    fpr, tpr, _ = roc_curve(y_valid_[:,1], y_pred_[:,1])
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr)
    
    roc_display.plot(ax = ax)
    ax.set_title(str(estimator_['clf']).split('(')[0] ,fontsize=12)
    
    #plt.close()
    
plt.show()

# 7 - Importância das Features

In [ ]:
pipeline_ = best_grid.best_estimator_
feature_names = pipeline_['preprocessor'].transformers_[1][1].get_feature_names(categorial_columns)

feature_names = list(feature_names)

for n in numerical_columns: 
    feature_names.append(n)

features_imp = []


for r in result:
    
    pipeline_ = r['grid'].best_estimator_
     
    if hasattr(pipeline_['clf'], 'feature_importances_'):
        
        
        clf_imp = pipeline_['clf']
        
        f_imp = pd.DataFrame(clf_imp.feature_importances_,index=feature_names, columns = [str(clf_imp).split('(')[0]])
        features_imp.append(f_imp)

dt_features_imp = pd.concat(features_imp, axis=1).sort_values(by=['RandomForestClassifier', 'XGBClassifier'], ascending=False)

dt_features_imp['DecisionTreeClassifier'] = dt_features_imp['DecisionTreeClassifier'].replace({0:np.nan})
dt_features_imp['ExtraTreeClassifier']    = dt_features_imp['ExtraTreeClassifier'].replace({0:np.nan})
dt_features_imp['RandomForestClassifier'] = dt_features_imp['RandomForestClassifier'].replace({0:np.nan})
dt_features_imp['XGBClassifier']          = dt_features_imp['XGBClassifier'].replace({0:np.nan})

dt_features_imp[0:30]

De acordo com a análise de feature importance, observamos que a maioria das features possui baixa relevância para o sucesso do modelo, sendo possível realizar uma redução de dimensão das features. No caso do XGBClassifier, por exemplo, 3 features (epoch_mjd, i, epoch) somaram mais de 99% de importância.

Procedendo somente com esse modelo (devido a ser considerado um state-of-the-art model, vencedor de diversas competições kaggle inclusive), resolvemos proceder redução de features, utilizando as 11 features mais importantes para o modelo:

In [ ]:
X = pd.concat((data['epoch_mjd'],
               data['i'],
               data['epoch'],
               data['tp'],
               data['H'],
               data['epoch_cal'],
               data['q'],
               data['sigma_e'],
               data['sigma_i'],
               data['per'],
               data['sigma_ma']
              ),axis=1)
y = data['class']

model = XGBClassifier()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0,
                                                        train_size=0.1, stratify=y)
        
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred = pd.DataFrame(y_pred,columns=['class'])

precisao = metrics.balanced_accuracy_score(y_valid, y_pred)
print(f'\nA precisão do modelo é de {(precisao*100).round(3)}%')